In [1]:
# importing Required Libraries
import numpy as np
import pandas as pd
import ast
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import requests
import re
# pd.set_option('max_colwidth', None)

In [2]:
#loading data
internal_data = pd.read_csv("internal_file_data.csv")

In [3]:
internal_data.head()

,Unnamed: 0,name,Wealth_History,personal_data
0,0,Elon Musk,"[{""percent"":3.8356164383561646,""worth"":""$8.4B""...","Age52Source of WealthTesla, SpaceX, Self MadeS..."
1,1,Bernard Arnault & family,"[{""percent"":15.876777251184834,""worth"":""$33.5B...","Age74Source of WealthLVMHResidenceParis, Franc..."
2,2,Jeff Bezos,"[{""percent"":18.07909604519774,""worth"":""$32B"",""...","Age59Source of WealthAmazon, Self MadeSelf-Mad..."
3,3,Larry Ellison,"[{""percent"":44.85981308411215,""worth"":""$48B"",""...","Age78Source of WealthOracle, Self MadeSelf-Mad..."
4,4,Bill Gates,"[{""percent"":58.91472868217055,""worth"":""$76B"",""...","Age67Source of WealthMicrosoft, Self MadeSelf-..."


In [4]:
internal_data.shape

(2609, 4)

In [5]:
internal_data.isnull().sum()

Unnamed: 0          0
name                3
Wealth_History    223
personal_data       3
dtype: int64

In [6]:
internal_data[internal_data['name'].isnull()]

,Unnamed: 0,name,Wealth_History,personal_data
1992,1992,NaN,NaN,NaN
2386,2386,NaN,NaN,NaN
2575,2575,NaN,NaN,NaN


## Summary for the Data

This is a Dataset containing information of billionaires from all over the world with more than 2600 rows mentioned on **realtime forbes billinaires list of 2023**. This dataset has three columns, 'Name' of the billionaires, their wealth history of last 6 to 8 years, and their personal information like citizenship, married or something else, and their Education in "personal_data" column.


### Column's Description

- `Name`: Name of the billionaires.

- `Wealth_History`: This column holds the information of billionaires's worth value(in billion dollar) in a particular year. Ex - ([Elon Musk] - (2014, 8.4B),(2015,12B),........

- `personal_data`: This column has many information about billionaires. What and Where did they study?, Where do they live? The company was made by him/her or inherited. Etc..

#### As this data is too messey and dirty, Further I will be doing data cleaning.

#### Issues With the Dataset

- `Unnamed: 0` - This column is of no use.

- `Name` - In name column, some rows contains **& family** with their name in last and some nan values are also there. - `Dirty Data`

- `Wealth_History` - This column contains wealth value of particular year between 2011 to 2023 in the form of dictionary and some nan values are also there. - `Messy Data` 

- `personal_data` - This column has many information, like Education, Citizenship etc, all in one column. - `Messy Data`

## Data Cleaning - Part 1

- Remove **Unnamed: 0** column
- Remove **& family** from name column, 3 values are nan in name column and also there are nan in corresponding column, So drop nan values.
- Extract wealth value with corresponding year value in tuple from wealth_history column. Do not drop nan values.
- Personal_data column has many information, Extract **Citizenship, Marital Status, Education** and create is coulumn **Empire** that will tell, Business was self made or something else.
- Some of the billionaires educations are multiple, Extract first Education and its College/Univrsity.

### Remove Unnamed: 0 column

In [7]:
internal_data.drop(columns = 'Unnamed: 0', inplace=True)

### Remove & family from name column, 3 values are nan in name column and also there are nan in corresponding column, So drop nan values.

In [8]:
internal_data[internal_data['name'].str.contains('& family', na=False)]

,name,Wealth_History,personal_data
1,Bernard Arnault & family,"[{""percent"":15.876777251184834,""worth"":""$33.5B...","Age74Source of WealthLVMHResidenceParis, Franc..."
8,Carlos Slim Helu & family,"[{""percent"":77.41935483870968,""worth"":""$72B"",""...","Age83Source of WealthTelecom, Self MadeResiden..."
14,Francoise Bettencourt Meyers & family,"[{""percent"":52.422360248447205,""worth"":""$42.2B...","Age70Source of WealthL'OréalResidenceParis, Fr..."
21,Julia Koch & family,"[{""percent"":63.66666666666667,""worth"":""$38.2B""...",Age61Source of WealthKoch IndustriesSelf-Made ...
22,David Thomson & family,"[{""percent"":41.544117647058826,""worth"":""$22.6B...","Age66Source of WealthMediaResidenceToronto, Ca..."
...,...,...,...
2553,Pu Zhongjie & family,"[{""percent"":77.77777777777779,""worth"":""$1.4B"",...","Age60Source of WealthMedical equipment, Self M..."
2570,David Tran & family,NaN,"Age77Source of WealthHot sauce, Self MadeResid..."
2573,Kangbao Ke & family,"[{""percent"":100,""worth"":""$1.9B"",""name"":""Billio...","Age60Source of WealthRetailing, Self MadeResid..."
2589,Petter Stordalen & family,"[{""percent"":77.77777777777779,""worth"":""$1.4B"",...","Age60Source of WealthHotels, Self MadeResidenc..."


In [9]:
internal_data['name'] = internal_data['name'].str.replace('& family','').str.strip()

In [10]:
internal_data[internal_data['name'].isnull()]
# Will drop three based on index value

,name,Wealth_History,personal_data
1992,NaN,NaN,NaN
2386,NaN,NaN,NaN
2575,NaN,NaN,NaN


In [11]:
internal_data.drop([1992, 2386, 2575], inplace=True)

In [12]:
internal_data

,name,Wealth_History,personal_data
0,Elon Musk,"[{""percent"":3.8356164383561646,""worth"":""$8.4B""...","Age52Source of WealthTesla, SpaceX, Self MadeS..."
1,Bernard Arnault,"[{""percent"":15.876777251184834,""worth"":""$33.5B...","Age74Source of WealthLVMHResidenceParis, Franc..."
2,Jeff Bezos,"[{""percent"":18.07909604519774,""worth"":""$32B"",""...","Age59Source of WealthAmazon, Self MadeSelf-Mad..."
3,Larry Ellison,"[{""percent"":44.85981308411215,""worth"":""$48B"",""...","Age78Source of WealthOracle, Self MadeSelf-Mad..."
4,Bill Gates,"[{""percent"":58.91472868217055,""worth"":""$76B"",""...","Age67Source of WealthMicrosoft, Self MadeSelf-..."
...,...,...,...
2604,Chang Jing,"[{""percent"":100,""worth"":""$2.5B"",""name"":""Billio...","Age40Source of WealthTechnology, Self MadeResi..."
2605,Vijay Shekhar Sharma,"[{""percent"":50,""worth"":""$1.3B"",""name"":""Billion...","Age45Source of Wealthfinancial technology, Sel..."
2606,Kanye West,"[{""percent"":90,""worth"":""$1.8B"",""name"":""Billion...","Age46Source of Wealthmusic, sneakers, Self Mad..."
2607,Sam Bankman-Fried,"[{""percent"":36.25,""worth"":""$8.7B"",""name"":""Bill...","Age31Source of Wealthcryptocurrency exchange, ..."


### Extract wealth value with corresponding year value in tuple from wealth_history column.

In [13]:
# Removing [ , ] brackets so that I could get dictionaries inside it.
internal_data['Wealth_History'] = internal_data['Wealth_History'].str.replace("[",'').str.replace("]",'')

In [14]:
internal_data['Wealth_History']

0       {"percent":3.8356164383561646,"worth":"$8.4B",...
1       {"percent":15.876777251184834,"worth":"$33.5B"...
2       {"percent":18.07909604519774,"worth":"$32B","n...
3       {"percent":44.85981308411215,"worth":"$48B","n...
4       {"percent":58.91472868217055,"worth":"$76B","n...
                              ...                        
2604    {"percent":100,"worth":"$2.5B","name":"Billion...
2605    {"percent":50,"worth":"$1.3B","name":"Billiona...
2606    {"percent":90,"worth":"$1.8B","name":"Billiona...
2607    {"percent":36.25,"worth":"$8.7B","name":"Billi...
2608    {"percent":100,"worth":"$4.5B","name":"Billion...
Name: Wealth_History, Length: 2606, dtype: object

In [15]:
# Extracting wealth value with corresponding year in tuple.
wealth = []
for i in internal_data['Wealth_History']:
    individuaal_wealth = []
    if pd.isna(i):
        wealth.append(np.nan)
    else:
        x = ast.literal_eval(i)
        for j in x:
            worth = j['worth'].replace('$','')
            date = j['date'].split(' ')[1]
            value = (date, worth)
            individuaal_wealth.append(value)
        wealth.append(individuaal_wealth)     

In [16]:
internal_data['Wealth_History'] = wealth

### Extract Citizenship from personal_data column.

In [17]:
internal_data['Citizenship'] = internal_data['personal_data'].str.split('Citizenship').str.get(1).str.split('Marital').str.get(0)
internal_data['Citizenship'] = internal_data['Citizenship'].str.split('Education').str.get(0)
internal_data['Citizenship'] = internal_data['Citizenship'].str.split('Child').str.get(0)

### Extract Marital Status from personal_data column.

In [18]:
internal_data['Marital Status'] = internal_data['personal_data'].str.split('Marital Status').str.get(1).str.split('Children').str.get(0)
internal_data['Marital Status'] = internal_data['Marital Status'].str.split('Education').str.get(0)

### Extract First Education from personal_data column.

In [19]:
internal_data['Education'] = internal_data['personal_data'].str.split('Education').str.get(1).str.split(',').str.get(0)

### Extract First College/University from personal_data column.

In [20]:
internal_data['College/University'] = internal_data['personal_data'].str.split('Education').str.get(1).str.split(';').str.get(0)

In [21]:
internal_data['College/University'] = internal_data['College/University'].str.split(',').str.get(-1).str.strip()

### Extract 'Self-made' value from personal_data column.

In [37]:
Empire = internal_data['personal_data'].str.split('Residence').str.get(0)

In [39]:
empire = []
for i in Empire:
    if 'Self Made' in i:
        empire.append("Self Made")
    else:
        empire.append(np.nan)

In [40]:
internal_data['Empire'] = empire

##### Now I have extracted all the required information from **Wealth History** and **Personal Data** Column. But There are still many issues in new extracted column. 
In this Data Cleaning - Part 2, I will be cleaning messy and dirty data from these columns.

### Isuues with this Dataset.

- Column `Wealth_History`, `Citizenship`, `Education`, `College/University` have some messy or dirty data.

## Data Cleaning - Part 2

- `CitizenShip Column`
    1. Index **[685]** has **United StatesAgentDavid FalkAgencyFAME** --- Replace with **United States**
    
    2. Index **[2512]** has **United StatesAgentMark SteinbergAgencyExcel Sports ManagementSponsors2K Sports, Autograph, Bridgestone, Full Swing, Kowa Company Ltd., Monster Energy, Rolex, TaylorMade, Upper Deck** --- Replace with **United States**


- `Education Column`   
    1. Index **[1904]** has **Xiamen University** --- Replace with **NaN**.
    2. Index **[976]** has **Empty** --- Replace with **NaN**.
    3. Index **[1858]** has **Empty** --- Replace with **Bachelor of Arts/Science**.
    4. Index **[2044]** has **Empty** --- Replace with **EMBA**.
    5. Index **[2144]** has **Empty** --- Replace with **NaN**.
    6. Index **[2501]** has **Empty** --- Replace with **NaN**.
    7. Index **[361]** has **Houston Community College** --- Replace with **NaN**.
    8. Index **[744]** has **Chinese Academy of Social Sciences; Bachelor of Arts/Science** --- Replace with **Bachelor of Arts/Science**.
    9. Index **[764]** has **Chinese Culture University** --- Replace with **NaN**.
    10. Index **[1030]** has **China Europe International Business School** --- Replace with **NaN**.
    11. Index **[1919]** has **London School of Economics; Master** --- Replace with **NaN**.
    12. Index **[2332]** has **University of Pennsylvania** --- Replace with **NaN**.
    13. Index **[2338]** has **Waseda University** --- Replace with **NaN**.
    14. Index **[1470]** has **Harvard Business School; Bachelor of Arts/Science** --- Replace with **NaN**.
    15. Index **[2160]** has **Oregon State University; Walla Walla College** --- Replace with **NaN**.
    16. Index **[339]** has **National Taipei University of Technology** --- Replace with **NaN**.
    17. Index **[2351]** has **University of California** --- Replace with **NaN**.
    18. Index **[279]** has **State Central Institute of Physical Culture** --- Replace with **NaN**.
    19. Index **[275]** has **Liaoning University** --- Replace with **NaN**.
    20. Index **[1224]** has **Fordham University; Bachelor of Arts/Science** --- Replace with **NaN**.
    21. Index **[1403]** has **City University of New York Baruch** --- Replace with **NaN**.
    22. Index **[1957]** has **Chinese University of Hong Kong** --- Replace with **NaN**.
    23. Index **[994]** has **ResidenceDubai** --- Replace with **NaN**.
    24. Index **[333]** has **Taipei University of Marine Technology** --- Replace with **NaN**.
    25. Index **[1126]** has **ResidenceBangalore** --- Replace with **NaN**.
    26. Index **[1855]** has **ResidencePhoenix** --- Replace with **Bachelor of Arts/Science**.
    27. Index **[1596]** has **Jiangsu Province Wujin Teacher School** --- Replace with **NaN**.
    28. Index **[1581]** has **Wharton School at the University of Pennsylvania** --- Replace with **NaN**.
    29. Index **[1617]** has **National Taitung University** --- Replace with **NaN**.
    30. Index **[2211]** has **Sweet Briar College** --- Replace with **NaN**.
    31. Index **[2212]** has **The University of North Carolina at Chapel Hill** --- Replace with **NaN**.
    32. Index **[2512]** has **Stanford University** --- Replace with **NaN**
    
    
    
- `College/University Column` 

    1. Index **[1215]** has **India** --- Replace with **Anna University**.
    2. Index **[189, 235, 661, 708, 711, 754, 1348, 2351, 58, 443, 662, 1697, 2174, 2293, 2325, 2342, 2601, 2028]** has **Los Angeles or Barkeley** --- Replace with **University of California**.
    3. Index **[1576]** has **Davis** --- Replace with **University of California**.
    4. Index **[193]** has **Austin** --- Replace with **University of Texas**.
    5. Index **[2032]** has **Kazan** --- Replace with **Moscow Power Engineering Institute**.
    6. Index **[91]** has **Eau Claire** --- Replace with **University of Wisconsin**.
    7. Index **[1133, 1170, 1372]** has **Ahmedabad** --- Replace with **Indian Institute of Management**.
    8. Index **[613]** has **Moscow)** --- Replace with **State Institute of Theatre Arts**.
    9. Index **[1368]** has **Electronics and Automation** --- Replace with **Moscow Institute of Radioengineering**.
    10. Index **[1798]** has **Madison** --- Replace with **University of Wisconsin**.
    11. Index **[1805]** has **marketing/management** --- Replace with **Stockholm University**.
    12. Index **[266]** has **Prague** --- Replace with **Czech Technical University**.
    13. Index **[1529]** has **Chapel Hill** --- Replace with **University of North Carolina**.
    14. Index **[170]** has **San Diego** --- Replace with **University of California**.
    15. Index **[234]** has **BHU** --- Replace with **Indian Institute of Technology**.
    16. Index **[1620]** has **Germany** --- Replace with **Rheinisch-Westfälische Technische Hochschule**.
    17. Index **[1693]** has **The Netherlands** --- Replace with **Erasmus University Rotterdam**.
    18. Index **[1960]** has **Bombay** --- Replace with **Indian Institute of Technology**.
    19. Index **[2052]** has **Jerusalem** --- Replace with **Hebrew University**.
    20. Index **[1132, 1139]** has **Lebanon** --- Replace with **American University of Beirut**.
    21. Index **[130]** has **Ceibs** --- Replace with **NaN**
    22. Index **[357]** has **London** --- Replace with **Sir John Cass College**
    23. Index **[389]** has **Sydney** --- Replace with **University of Technology**
    24. Index **[479]** has **Cambridge** --- Replace with **University College London**
    25. Index **[976, 994, 1126, 1855, 1858, 2144, 2501]** has **Wrong Value** --- Replace with **NaN**
    26. Index **[2044]** has **Wrong Value** --- Replace with **Europe International Business School**
    27. Index **[2143]** has **India** --- Replace with **Indian Institute of Technology (IIT) Kharagpur**

Some values are dirty(should not be there) those dirty data are mentioned above. Along with that Some of the Colleges/Universities name are not written in English Language rather they are written in their native lanuguage. So I am leaving those names as it is.
   
   
- `Wealth_History Column`   

Create Seperate Column by Year name that will have the worth value if available in wealth history column else NaN


#### Drop Wealth_History and Personal_data column.

- After Creating seperate column from 2014 to 2023, all worth values are in billions, but index **[2563]'s** **2015**, and **2016** values are in million. So, convert it billion. Then **Replace 'B',''** and covert astype to **float**

### Cleaning Citizenship Column

In [43]:
internal_data['Citizenship'].unique()

array(['United States', 'France', 'Mexico', 'India', 'Spain', 'China',
       'Canada', 'Germany', 'Austria', 'Italy', 'Hong Kong', 'Japan',
       'Switzerland', 'Australia', 'Indonesia', 'Russia', 'Chile',
       'United Kingdom', 'Israel', 'Sweden', 'Brazil', 'Czech Republic',
       'Singapore', 'Thailand', 'Netherlands', 'Cyprus', 'South Africa',
       'United Arab Emirates', 'Malaysia', 'Nigeria', 'Philippines',
       'South Korea', 'New Zealand', 'Taiwan', 'Belgium', 'Monaco',
       'Colombia', 'Norway', 'Ireland', 'Denmark', 'Egypt', 'Poland',
       'Eswatini (Swaziland)', 'Ukraine', 'Greece', 'Georgia', 'Turkey',
       'Portugal', 'Argentina', 'Kazakhstan', 'Algeria', 'Venezuela',
       'United StatesAgentDavid FalkAgencyFAME', 'Finland', 'Vietnam',
       'Belize', 'Lebanon', 'Iceland', 'Guernsey', 'Liechtenstein',
       'Bulgaria', 'Oman', 'Romania', 'Zimbabwe', 'Nepal', 'Qatar',
       'Slovakia', 'Morocco', 'Hungary', 'Peru', 'Estonia', 'Tanzania',
       'Barbados'

In [44]:
internal_data.loc[[685,2512], 'Citizenship'] = 'United States'

In [45]:
internal_data['Citizenship'].unique()

array(['United States', 'France', 'Mexico', 'India', 'Spain', 'China',
       'Canada', 'Germany', 'Austria', 'Italy', 'Hong Kong', 'Japan',
       'Switzerland', 'Australia', 'Indonesia', 'Russia', 'Chile',
       'United Kingdom', 'Israel', 'Sweden', 'Brazil', 'Czech Republic',
       'Singapore', 'Thailand', 'Netherlands', 'Cyprus', 'South Africa',
       'United Arab Emirates', 'Malaysia', 'Nigeria', 'Philippines',
       'South Korea', 'New Zealand', 'Taiwan', 'Belgium', 'Monaco',
       'Colombia', 'Norway', 'Ireland', 'Denmark', 'Egypt', 'Poland',
       'Eswatini (Swaziland)', 'Ukraine', 'Greece', 'Georgia', 'Turkey',
       'Portugal', 'Argentina', 'Kazakhstan', 'Algeria', 'Venezuela',
       'Finland', 'Vietnam', 'Belize', 'Lebanon', 'Iceland', 'Guernsey',
       'Liechtenstein', 'Bulgaria', 'Oman', 'Romania', 'Zimbabwe',
       'Nepal', 'Qatar', 'Slovakia', 'Morocco', 'Hungary', 'Peru',
       'Estonia', 'Tanzania', 'Barbados', 'St. Kitts and Nevis',
       'Armenia', 'Macau

### Cleaning Education Column

In [46]:
internal_data['Education'].unique()

array(['Bachelor of Arts/Science', 'Drop Out', 'Master of Science',
       'Master of Business Administration', nan,
       'Doctor of Jurisprudence', 'Master of Arts',
       'Bachelor of Engineering', 'Diploma',
       'Bachelor of Science in Engineering', 'Bachelor of Science',
       'Doctorate', 'Associate in Arts/Science', 'Master',
       'Masters in Finance', 'Master of Science in Engineering',
       'Bachelor of Technology', 'EMBA', 'Master of Laws', 'Ph.D',
       'Master of Economics', 'High School Graduate',
       'Bachelor of Arts/Economics', 'Medical Doctor', 'LLB',
       'Law Degree', 'Graduate', 'Registered Nurse',
       'Liaoning University',
       'State Central Institute of Physical Culture',
       'Master of Arts/Science', 'Taipei University of Marine Technology',
       'National Taipei University of Technology',
       'Houston Community College', 'Bachelor of Business Administration',
       'Bachelor of Commerce', 'Masters of Public Health',
       'Doctor

In [47]:
internal_data.loc[[1904, 976, 2144, 2501, 361, 764, 1030, 1919, 2332, 2338, 1470, 2160, 339, 2351, 279, 275, 1224, 1403, 1957, 994, 333, 1126, 1596, 1581, 1617, 2211, 2212, 2512], 
'Education'] = np.nan

In [48]:
internal_data.loc[[1858, 744, 1855], 'Education'] = ' Bachelor of Arts/Science'

In [49]:
internal_data.loc[2044, 'Education'] = 'EMBA'

In [50]:
internal_data['Education'].unique()

array(['Bachelor of Arts/Science', 'Drop Out', 'Master of Science',
       'Master of Business Administration', nan,
       'Doctor of Jurisprudence', 'Master of Arts',
       'Bachelor of Engineering', 'Diploma',
       'Bachelor of Science in Engineering', 'Bachelor of Science',
       'Doctorate', 'Associate in Arts/Science', 'Master',
       'Masters in Finance', 'Master of Science in Engineering',
       'Bachelor of Technology', 'EMBA', 'Master of Laws', 'Ph.D',
       'Master of Economics', 'High School Graduate',
       'Bachelor of Arts/Economics', 'Medical Doctor', 'LLB',
       'Law Degree', 'Graduate', 'Registered Nurse',
       'Master of Arts/Science', 'Bachelor of Business Administration',
       'Bachelor of Commerce', 'Masters of Public Health',
       'Doctorate in Pedagogical Sciences', 'LLM',
       'Bachelor of Social Sciences', 'Master of Philosophy',
       ' Bachelor of Arts/Science', 'Master of Engineering Management',
       'B.Com', 'Bachelor of Law', 'Charte

### Cleaning College/University Column

In [51]:
internal_data['College/University'].unique()

array(['University of Pennsylvania', 'Ecole Polytechnique de Paris',
       'Princeton University', 'University of Chicago',
       'Harvard University', 'Columbia University',
       'Universidad Nacional Autonoma de Mexico', 'Stanford University',
       'Harvard Business School', nan, 'University of Arkansas',
       'Trinity University', 'The University of Texas at Austin',
       'Massachusetts Institute of Technology',
       'University of Central Arkansas', 'University of Cambridge',
       'Nankai University', 'Stanford Graduate School of Business',
       'Bryn Mawr College', 'The Hotchkiss School',
       'Oregon State University', 'High School', 'Waseda University',
       'Hebrew University Jerusalem', 'Shenzhen University',
       'Chinese Academy of Social Sciences',
       'University of Electronic Science and Technology of China',
       'SUNY Binghamton', 'Pune University', 'New York University',
       'Technical University of Karlsruhe', 'Berkeley',
       'PSG Coll

In [52]:
# Filling these index values with 'University of California'
internal_data.loc[[189, 235, 661, 708, 711, 754, 1348, 2351, 1576, 170, 58, 443, 662, 1697, 2174, 2293, 2325, 2342, 2601, 2028], 'College/University'] = 'University of California'

In [53]:
# Filling these index with 'Indian Institute of Technology'
internal_data.loc[[234, 1960], 'College/University'] = 'Indian Institute of Technology'

In [54]:
internal_data.loc[[1215, 193, 2032, 91, 613, 1368, 1798, 1805, 266, 1529, 1620, 1693, 2052, 357, 389, 479, 2044, 2143], 'College/University'] = [
 'Anna University', 'University of Texas', 'Moscow Power Engineering Institute', 'University of Wisconsin',
 'State Institute of Theatre Arts', 'Moscow Institute of Radioengineering', 'University of Wisconsin',
 'Stockholm University', 'Czech Technical University', 'University of North California',
 'Rheinisch-Westfälische Technische Hochschule', 'Erasmus University Rotterdam', 'Hebrew University',
 'Sir John Cass College', 'University of Technology', 'University College London', 'Europe International Business School',
 'Indian Institute of Technology (IIT) Kharagpur']

In [55]:
internal_data.loc[[1133, 1170, 1372], 'College/University'] = 'Indian Institute of Management'

In [56]:
internal_data.loc[[1132, 1139], 'College/University'] = 'American University of Beirut'

In [57]:
# Replaceing Dirty Data with NaN
internal_data.loc[[130, 976, 994, 1126, 1855, 1858, 2144, 2501], 'College/University'] = np.nan

In [58]:
internal_data['College/University'].unique()

array(['University of Pennsylvania', 'Ecole Polytechnique de Paris',
       'Princeton University', 'University of Chicago',
       'Harvard University', 'Columbia University',
       'Universidad Nacional Autonoma de Mexico', 'Stanford University',
       'Harvard Business School', nan, 'University of Arkansas',
       'Trinity University', 'The University of Texas at Austin',
       'Massachusetts Institute of Technology',
       'University of Central Arkansas', 'University of Cambridge',
       'Nankai University', 'Stanford Graduate School of Business',
       'Bryn Mawr College', 'The Hotchkiss School',
       'Oregon State University', 'High School', 'Waseda University',
       'Hebrew University Jerusalem', 'Shenzhen University',
       'Chinese Academy of Social Sciences',
       'University of Electronic Science and Technology of China',
       'SUNY Binghamton', 'Pune University', 'New York University',
       'Technical University of Karlsruhe', 'University of California',


In [59]:
internal_data.reset_index(inplace=True)
internal_data.drop(columns=['index'], inplace=True)

### Cleaning Wealth History Column

In [60]:
# Creating Seperate Column by Year name that will have the worth value if available in wealth history column else NaN

In [61]:
def get_worth(year):
    l = []
    for i in internal_data['Wealth_History']:
        if type(i) == float:
            l.append({np.nan})
        if type(i) != float:
            s = {np.nan}
            for j in i:
                if year in j[0]:
                    s.add(j[1])
                else:
                    s.add(np.nan)
            l.append(s)   
        
    return l              

In [62]:
def fetch_worth(year):
    worth_value = []
    for i in internal_data[year]:
        found = False
        for element in i:
            if isinstance(element, str):
                worth_value.append(element)
                found = True
        if not found:
            worth_value.append(np.nan)
            
    return worth_value                       

#### Creating Year 2014

In [63]:
internal_data['2014'] = get_worth('2014')
internal_data['2014'] = fetch_worth('2014')

#### Creating Year 2015

In [64]:
internal_data['2015'] = get_worth('2015')
internal_data['2015'] = fetch_worth('2015')

#### Creating Year 2016

In [65]:
internal_data['2016'] = get_worth('2016')
internal_data['2016'] = fetch_worth('2016')

#### Creating Year 2017

In [66]:
internal_data['2017'] = get_worth('2017')
internal_data['2017'] = fetch_worth('2017')

#### Creating Year 2018

In [67]:
internal_data['2018'] = get_worth('2018')
internal_data['2018'] = fetch_worth('2018')

#### Creating Year 2019

In [68]:
internal_data['2019'] = get_worth('2019')
internal_data['2019'] = fetch_worth('2019')

#### Creating Year 2020

In [69]:
internal_data['2020'] = get_worth('2020')
internal_data['2020'] = fetch_worth('2020')

#### Creating Year 2021

In [70]:
internal_data['2021'] = get_worth('2021')
internal_data['2021'] = fetch_worth('2021')

#### Creating Year 2022

In [71]:
internal_data['2022'] = get_worth('2022')
internal_data['2022'] = fetch_worth('2022')

#### Creating Year 2023

In [72]:
internal_data['2023'] = get_worth('2023')
internal_data['2023'] = fetch_worth('2023')

In [73]:
internal_data

,name,Wealth_History,personal_data,Citizenship,Marital Status,Education,College/University,Empire,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Elon Musk,"[(2014, 8.4B), (2015, 12B), (2016, 10.7B), (20...","Age52Source of WealthTesla, SpaceX, Self MadeS...",United States,Single,Bachelor of Arts/Science,University of Pennsylvania,Self Made,8.4B,12B,10.7B,13.9B,19.9B,22.3B,24.6B,151B,219B,180B
1,Bernard Arnault,"[(2014, 33.5B), (2015, 37.2B), (2016, 34B), (2...","Age74Source of WealthLVMHResidenceParis, Franc...",France,Married,Bachelor of Arts/Science,Ecole Polytechnique de Paris,NaN,33.5B,37.2B,34B,41.5B,72B,76B,76B,150B,158B,211B
2,Jeff Bezos,"[(2014, 32B), (2015, 34.8B), (2016, 45.2B), (2...","Age59Source of WealthAmazon, Self MadeSelf-Mad...",United States,Engaged,Bachelor of Arts/Science,Princeton University,Self Made,32B,34.8B,45.2B,72.8B,112B,131B,113B,177B,171B,114B
3,Larry Ellison,"[(2014, 48B), (2015, 54.3B), (2016, 43.6B), (2...","Age78Source of WealthOracle, Self MadeSelf-Mad...",United States,In Relationship,Drop Out,University of Chicago,Self Made,48B,54.3B,43.6B,52.2B,58.5B,62.5B,59B,93B,106B,107B
4,Bill Gates,"[(2014, 76B), (2015, 79.2B), (2016, 75B), (201...","Age67Source of WealthMicrosoft, Self MadeSelf-...",United States,Divorced,Drop Out,Harvard University,Self Made,76B,79.2B,75B,86B,90B,96.5B,98B,124B,129B,104B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2601,Chang Jing,"[(2021, 2.5B), (2022, 1.3B), (2023, 1.1B)]","Age40Source of WealthTechnology, Self MadeResi...",China,NaN,NaN,NaN,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5B,1.3B,1.1B
2602,Vijay Shekhar Sharma,"[(2017, 1.3B), (2018, 1.7B), (2019, 2.6B), (20...","Age45Source of Wealthfinancial technology, Sel...",India,Married,Bachelor of Science in Engineering,Delhi University,Self Made,NaN,NaN,NaN,1.3B,1.7B,2.6B,2.4B,2.3B,1.2B,NaN
2603,Kanye West,"[(2021, 1.8B), (2022, 2B)]","Age46Source of Wealthmusic, sneakers, Self Mad...",United States,Divorced,Drop Out,Chicago State University,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.8B,2B,NaN
2604,Sam Bankman-Fried,"[(2021, 8.7B), (2022, 24B)]","Age31Source of Wealthcryptocurrency exchange, ...",United States,NaN,Bachelor of Arts/Science,Massachusetts Institute of Technology,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7B,24B,NaN


In [74]:
internal_data.drop(columns=['Wealth_History', 'personal_data'], inplace=True)

In [75]:
# Convert 2015, and 2015 values in billion
internal_data[internal_data['2015'].str.contains('M', na=False)]

,name,Citizenship,Marital Status,Education,College/University,Empire,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
2563,Arash Ferdowsi,United States,Single,Drop Out,Massachusetts Institute of Technology,Self Made,NaN,500M,400M,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
internal_data.loc[2563, '2015'] = '0.5B'

In [77]:
internal_data.loc[2563, '2016']= '0.4B'

### Replacing 'B','' and converting type to float

In [78]:
internal_data['2014'] = internal_data['2014'].str.replace('B','').str.strip().astype(float)
internal_data['2015'] = internal_data['2015'].str.replace('B','').str.strip().astype(float)
internal_data['2016'] = internal_data['2016'].str.replace('B','').str.strip().astype(float)
internal_data['2017'] = internal_data['2017'].str.replace('B','').str.strip().astype(float)
internal_data['2018'] = internal_data['2018'].str.replace('B','').str.strip().astype(float)
internal_data['2019'] = internal_data['2019'].str.replace('B','').str.strip().astype(float)
internal_data['2020'] = internal_data['2020'].str.replace('B','').str.strip().astype(float)
internal_data['2021'] = internal_data['2021'].str.replace('B','').str.strip().astype(float)
internal_data['2022'] = internal_data['2022'].str.replace('B','').str.strip().astype(float)
internal_data['2023'] = internal_data['2023'].str.replace('B','').str.strip().astype(float)

In [79]:
internal_data.to_csv('billionaires_personal_data.csv', encoding='utf-8')